In [24]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as const
import scipy.integrate as intg
import IPython.display as disp
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

This notebook is intended to check the consistency between 3 different methods for calculating the absorption of a layer in S4.
The methods are as follows

1. Using the GetPowerFlux method and taking the difference between what enters the the layer and what leaves the the layer. 
2. Using the GetVolumeIntegral function
3. Integrate the raw field output numerically

All of these methods should agree with one another, so let's verify this. Below I just set up the sim so it's ready for all subsequent computations

In [45]:
conf = Config('GaAs_slab.yml')
sim = Simulator(conf)
sim = setup_sim(sim)

DEBUG:nanowire.optics.simulate:Logger initialized
DEBUG:nanowire.optics.simulate:Building layer: Air
DEBUG:nanowire.optics.simulate:Layer Order 1
DEBUG:nanowire.optics.simulate:Building layer: Substrate
DEBUG:nanowire.optics.simulate:Layer Order 3
DEBUG:nanowire.optics.simulate:Building layer: Below_Air
DEBUG:nanowire.optics.simulate:Layer Order 4
DEBUG:nanowire.optics.simulate:Physical Frequency = 4.000000E+14
DEBUG:nanowire.optics.simulate:[399723277333333.31, 400256953271028.0]
DEBUG:nanowire.optics.simulate:[2.3155394055977224e-12, 2.3123628270194848e-12]
INFO:nanowire.optics.simulate:Incident Intensity: 1.23490003178
DEBUG:nanowire.optics.simulate:Incident Power: 0.077181251986
DEBUG:nanowire.optics.simulate:Incident Amplitude: 21.5690583027


# Poynting Flux Method

So at every layer interface, we have both forward and backward modes that each individually carry power. 
The forward direction signifies waves moving downwards, and the backward direction signifies waves moving
upward. A quick note about the indices on the terms. The first index describes the direction (f = forward, b = backward) and the second index describes where we are in the layer (t = top, b = bottom).  Using this
convention, the total power absorbed should be 

\begin{equation}
    P_{abs} = P_{f,t} + P_{b, b} - (P_{b, t} + P_{f, b})
\end{equation}

When S4 returns these numbers, it sticks a minus sign in front of the backward components to signify they are
traveling in the negative z directon (upwards). So, we need to stick an extra minus sign in front of a few
terms as follows:

\begin{equation}
    P_{abs} = P_{f,t} + -1 \cdot P_{b, b} - (-1 \cdot P_{b, t} + P_{f, b})
\end{equation}

And this correctly describes the total amount of power entering minus the total amount of power leaving. I 
implement this calculation below.

In [46]:
fluxes = sim.get_fluxes()

DEBUG:nanowire.optics.simulate:Computing fluxes ...
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Below_Air
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Substrate
DEBUG:nanowire.optics.simulate:Computing fluxes through layer: Air
DEBUG:nanowire.optics.simulate:Finished computing fluxes!


In [47]:
summed_absorbed_power = 0
abs_dict_fluxmethod = {}
for layer, (forw_top, back_top) in fluxes.items():
    if '_bottom' in layer:
        continue
    bottom = layer+'_bottom'
    forw_bot, back_bot = fluxes[bottom] 
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Forward Top: {}'.format(forw_top))
    print('Backward Top: {}'.format(back_top))
    print('Forward Bottom: {}'.format(forw_bot))
    print('Backward Bottom: {}'.format(back_bot))
    P_in = forw_top + -1*back_bot
    P_out = forw_bot + -1*back_top
    print('Power Entering Layer: {}'.format(P_in))
    print('Power Leaving Layer: {}'.format(P_out))
    P_abs = P_in - P_out
    abs_dict_fluxmethod[layer] = P_abs
    print('Absorbed in Layer: {}'.format(P_abs))
    summed_absorbed_power += P_abs
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))
print('GaAs Absorption = {}'.format(abs_dict_fluxmethod['Substrate']))

-------------------------
Layer: Substrate
Forward Top: (21.973812057569923+0.14334866515010325j)
Backward Top: (-1.2349872518008014-0.14334866515010325j)
Forward Bottom: (8.90491046583286+0.14913344020036184j)
Backward Bottom: (-2.9496124861525645-0.14913344020036184j)
Power Entering Layer: (24.923424543722486+0.2924821053504651j)
Power Leaving Layer: (10.13989771763366+0.2924821053504651j)
Absorbed in Layer: (14.783526826088826+0j)
-------------------------
Layer: Below_Air
Forward Top: (5.955297979680294+0j)
Backward Top: 0j
Forward Bottom: (5.955297979680294+0j)
Backward Bottom: 0j
Power Entering Layer: (5.955297979680294+0j)
Power Leaving Layer: (5.955297979680294+0j)
Absorbed in Layer: 0j
-------------------------
Layer: Air
Forward Top: (25+0j)
Backward Top: (-4.261175194230889+0j)
Forward Bottom: (25+0j)
Backward Bottom: (-4.261175194230889+0j)
Power Entering Layer: (29.261175194230887+0j)
Power Leaving Layer: (29.261175194230887+0j)
Absorbed in Layer: 0j
----------------------

In [56]:
14.783526826088826**2

218.55266541768796

In [48]:
incident_amp_one_power_absorbed = .5913410730435529
incident_amp_two_power_absorbed = 2.3653642921742115
incident_amp_three_power_absorbed = 5.3220696573919755
incident_amp_four_power_absorbed = 9.461457168696846
incident_amp_five_power_absorbed = 14.783526826088826 
ratio_one2two = incident_amp_two_power_absorbed/incident_amp_one_power_absorbed
ratio_one2three = incident_amp_three_power_absorbed/incident_amp_one_power_absorbed
ratio_one2four = incident_amp_four_power_absorbed/incident_amp_one_power_absorbed
ratio_one2five = incident_amp_five_power_absorbed/incident_amp_one_power_absorbed
print(ratio_one2two)
print(ratio_one2three)
print(ratio_one2four)
print(ratio_one2five)

4.0
9.0
16.0
25.000000000000007


So this just tells us that the incident power is equal to the amplitude of the incident electric field squared.
This is what we would expect, because we know:

\begin{equation}
    S = \frac{1}{c \mu_o} \frac{|E|^2}{2}
\end{equation}

where $S$ is the magnitude of the Poynting vector. In Lorentz-Heaviside units (the units used by S4), $c = \mu_o = 1$, so this also tells us that the power 
values are indeed not time averaged and are missing that factor of .5. This also tells us something about what
the power flux function is giving us back!!!!!! It's either not integrating, or dividing by the area, or something else like that. If it were integrating a uniform poynting vector magnitude 

# GetVolumeIntegral Method

We use the provided S4 function to compute

\begin{equation}
    \int \epsilon | \vec{E} | ^2 dV
\end{equation}
  
(this is what `Quantity = "E"` means in the code below) where the integral is over the volume of the layer. The
total absorbed power is 

\begin{equation}
    P_{abs} = -\frac{\omega}{2} \int Im(\epsilon) | \vec{E} | ^2 dV
\end{equation}

Where here we need to convert $\omega$ into the Lorentz-Heaviside units used by S4. $|E|^2$ should be a purely real quantity,
meaning we can take the imaginary part of the result of the integration and multiply by $\omega / 2$ to get the absorbed 
power.

In [6]:
int_result = sim.s4.GetLayerVolumeIntegral(Layer="Substrate", Quantity="E")
int_result2 = sim.s4.GetLayerVolumeIntegral(Layer="Substrate", Quantity="e")
int_result3 = sim.s4.GetLayerVolumeIntegral(Layer="Substrate", Quantity="U")
epsilon = sim._get_epsilon(os.path.expandvars(
    '$HOME/software/nanowire/nanowire/NK/006_GaAs_nk_Walker_modified_Hz.txt'))
si_freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
o1 = disp.Latex('$\epsilon = {}$'.format(epsilon))
o2 = disp.Latex("SI $\omega = {}$".format(si_freq))
o3 = disp.Latex("LH $\omega = {}$".format(f_conv))
o4 = disp.Latex('$\int \epsilon |E|^2 dV = {}$'.format(int_result))
o5 = disp.Latex('$\int |E|^2 dV = {}$'.format(int_result2))
o52 = disp.Latex('$\epsilon \int |E|^2 dV = {}$'.format(epsilon*int_result2))
msg = "$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV = " + "{}$".format(.5*f_conv*epsilon.imag*int_result2)
o6 = disp.Latex(msg)
o8 = disp.Latex("$\int \epsilon |E|^2 + |H|^2 dV  = {}$".format(int_result3))
test = disp.display(o1, o2, o3, o4, o5, o52, o6, o8)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

So this is a bit odd. We would expect

\begin{equation}
    \int |E|^2 dV
\end{equation}

to be purely real, but it clearly is not. Don't know what to make of this.

If we do the same thing but using the Lua API and the latest version of S4 on the master branch we get
similiar results. The lua script is below.

In [7]:
%cat volume_integral.lua

S = S4.NewSimulation()
-- Params
period = .25
numbasis = 40
eps_real = 13.77
eps_imag = 0.8
thickness = .5
freq = 1.3342563807926082
--
S:SetVerbosity(9)
S:SetLattice({period,0}, {0,period})
S:SetNumG(numbasis)
S:AddMaterial('Vacuum', {1.0, 0})
S:AddMaterial('GaAs', {eps_real, eps_imag})
S:AddLayer('top', 0, 'Vacuum')
S:AddLayer('slab', thickness, 'GaAs')
S:AddLayerCopy('bottom', 0, 'top')
S:SetExcitationPlanewave(
        {0, 0}, -- phi in [0,180), theta in [0,360)
        {1, 0}, -- s-polarization amplitude and phase in degrees
        {0, 0}) -- p-polarization
S:SetFrequency(freq)
tfr, tbr, tfi, tbi = S:GetPowerFlux('top')
bfr, bbr, bfi, bbi = S:GetPowerFlux('bottom')
print("### Top ###")
print(tfr)
print(tbr)
print(tfi)
print(tbi)
print("### Bottom ###")
print(bfr)
print(bbr)
print(bfi)
print(bbi)
pabs_r = (tfr - bfr) + (tbr - bbr)
pabs_i = (tfi - bfi) + (tbi - bbi)
print("Flux Method Power Absorbed Real: " .. pabs_r)
print("Flux Method Power Absorbed Imag: " .. pabs_i)
int_r, int_

Here is what we get if we run it

In [8]:
output = !! $HOME/software/S4/build/S4 volume_integral.lua

In [9]:
print(output[-7])
print(output[-6])
luaapi_1 = disp.display_latex("$"+output[-2]+"$", raw=True)
luaapi_2 = disp.display_latex("$"+output[-1]+"$", raw=True)
luaapi_3 = disp.display_latex("$"+output[-4]+"$", raw=True)
luaapi_4 = disp.display_latex("$"+output[-3]+"$", raw=True)

Flux Method Power Absorbed Real: 0.59119167316067
Flux Method Power Absorbed Imag: 0


So the results seem to agree decently well between the Lua and Python API. However, regardless of which API
we use, it is troubling that the integral of the electric field intensity has imaginary components.

# Integrate Raw Fields

This approach collects the raw fields from S4 and integrates them over the volume of the layer. Here
we are computing 

\begin{equation}
    \int \epsilon | \vec{E} | ^2 dV
\end{equation}

Which can be broken into real and imaginary parts


\begin{equation}
    \int \epsilon | \vec{E} | ^2 dV = \int Re(\epsilon) | \vec{E} | ^2 dV + i\int Im(\epsilon) | \vec{E} | ^2 dV
\end{equation}

The second term is the one that corresponds to absorbed power, which is given by:

\begin{equation}
    P_{absorbed} = \frac{\omega}{2} \int Im(\epsilon) | \vec{E} | ^2 dV
\end{equation}

In [49]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness, zsamps)
    x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
    y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

In [50]:
def integrate_SI(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness*1e-6, zsamps)
    x_vals = np.linspace(0, sim_proc.period*1e-6, sim_proc.x_samples)
    y_vals = np.linspace(0, sim_proc.period*1e-6, sim_proc.y_samples)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

In [51]:
sim.get_field()

DEBUG:nanowire.optics.simulate:Computing fields ...
DEBUG:nanowire.optics.simulate:Computing up to depth of 0.5 microns
DEBUG:nanowire.optics.simulate:Finished computing fields!


In [52]:
epsilon = sim._get_epsilon(os.path.expandvars(
    '$HOME/software/nanowire/nanowire/NK/006_GaAs_nk_Walker_modified_Hz.txt'))
print("Epsilon = {}".format(epsilon))
si_freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
print("SI Freq = {}".format(si_freq))
print("LH Freq = {}".format(f_conv))

Epsilon = (13.767280000000001+0.8017920000000001j)
SI Freq = 400000000000000.0
LH Freq = 1.3342563807926082


In [53]:
sim_proc = Simulation(simulator=sim)
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
layer_name = 'Substrate'
layer_obj = sim_proc.layers[layer_name]
si_freq = sim_proc.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim_proc.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
n_mat, k_mat = layer_obj.get_nk_matrix(si_freq)
# n and k could be functions of space, so we need to multiply the
# fields by n and k before integrating
res = integrate_SI(Esq, layer_obj, sim_proc)
imag_part = res*(2*n_mat[0, 0]*k_mat[0, 0])
p_abs_imag = .5*si_freq*const.epsilon_0*imag_part
print(si_freq)
print(res)
print(f_conv)
print(const.c/si_freq*1e9)
disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

DEBUG:nanowire.optics.postprocess:Logger initialized
DEBUG:nanowire.optics.postprocess:Adding normEsquared to data dict


400000000000000.0
(1.37463654512e-19+0j)
1.3342563807926082
749.481145


In [54]:
print(np.average(Esq))

(4.41905227606+0j)


In [55]:
sim_proc = Simulation(simulator=sim)
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
print(Esq.dtype)
layer_name = 'Substrate'
layer_obj = sim_proc.layers[layer_name]
print("Layer : {}".format(layer_name))
si_freq = sim_proc.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim_proc.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
n_mat, k_mat = layer_obj.get_nk_matrix(si_freq)
print("n = {}".format(n_mat[0,0]))
print("k = {}".format(k_mat[0,0]))
raw_fields1 = disp.Latex("$Re(\epsilon) = {}$".format(n_mat[0, 0]**2 - k_mat[0, 0]**2))
raw_fields2 = disp.Latex("$Im(\epsilon) = {}$".format(2*n_mat[0,0]*k_mat[0,0]))
disp.display(o1, o2)
epsilon = complex(n_mat[0, 0]**2 - k_mat[0, 0]**2, 2*n_mat[0,0]*k_mat[0,0])
print(np.absolute(epsilon))
# n and k could be functions of space, so we need to multiply the
# fields by n and k before integrating
print('Slice: {}'.format(layer_obj.slice))
res = integrate(Esq, layer_obj, sim_proc)
raw_fields3 = disp.Latex("$\int |E|^2 dV = {}$".format(res))
disp.display(raw_fields3)
#arr_slice = Esq[layer_obj.slice]*(n_mat**2 - k_mat**2)
real_part = res*(n_mat[0, 0]**2 - k_mat[0, 0]**2)
imag_part = res*(2*n_mat[0, 0]*k_mat[0, 0])
#p_abs_imag = .5*f_conv*imag_part
#p_abs_imag = (1/16)*.5*si_freq*const.epsilon_0*imag_part
p_abs_imag = .5*si_freq*const.epsilon_0*imag_part
disp.display_latex("$Re(\epsilon) \int |E|^2 dV = {}$".format(real_part), raw=True)
disp.display_latex("$Im(\epsilon) \int |E|^2 dV = {}$".format(imag_part), raw=True)
disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
print(n_mat[0, 0])
print(const.epsilon_0)
print('{:E}'.format(si_freq))

DEBUG:nanowire.optics.postprocess:Logger initialized
DEBUG:nanowire.optics.postprocess:Adding normEsquared to data dict


complex128
Layer : Substrate
n = 3.7125717990361693
k = 0.108


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

13.794853363
Slice: (slice(0, 150, None), Ellipsis)


<IPython.core.display.Latex object>

3.71257179904
8.854187817620389e-12
4.000000E+14


In [17]:
fields = np.absolute(sim.data["Ex"])**2 + np.absolute(sim.data['Ey'])**2
result = integrate(fields, layer_obj, sim_proc)
print(result)

0.0879767388875


In [18]:
fields = np.absolute(sim.data['Ez'])**2
result = integrate(fields, layer_obj, sim_proc)
print(result)

8.07958682619e-34


# Conclusion

## Flux Method

In [19]:
print('Python API:\n')
print('GaAs Absorption = {}'.format(abs_dict_fluxmethod['Substrate']))
print("-"*25)
print("Lua API:\n")
print(output[-7])
print(output[-6])

Python API:

GaAs Absorption = (0.5913410730435529+0j)
-------------------------
Lua API:

Flux Method Power Absorbed Real: 0.59119167316067
Flux Method Power Absorbed Imag: 0


## GetVolumeIntegral Method

In [20]:
print('Python API:\n')
test = disp.display(o4, o5, o52, o6, o8)
print("-"*25)
print("Lua API:" )
luaapi_1 = disp.display_latex("$"+output[-2]+"$", raw=True)
luaapi_2 = disp.display_latex("$"+output[-1]+"$", raw=True)
luaapi_3 = disp.display_latex("$"+output[-4]+"$", raw=True)
luaapi_4 = disp.display_latex("$"+output[-3]+"$", raw=True)

Python API:



<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

-------------------------
Lua API:


## Integrate Raw Fields

In [21]:
raw_fields3 = disp.Latex("$\int |E|^2 dV = {}$".format(res))
disp.display(raw_fields3)
disp.display_latex("$Re(\epsilon) \int |E|^2 dV = {}$".format(real_part), raw=True)
disp.display_latex("$Im(\epsilon) \int |E|^2 dV = {}$".format(imag_part), raw=True)
disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

<IPython.core.display.Latex object>

In [22]:
abs_dict_fluxmethod['Substrate']/p_abs_imag

(0.0047332870945384509+0j)